In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict
import re
# Đọc dữ liệu từ file parquet
train_df = pd.read_parquet("/kaggle/input/mambaa/train-00000-of-00001 (2).parquet")
train_df = train_df[train_df['text'].str.strip() != ""]
val_df = pd.read_parquet("/kaggle/input/mambaa/validation-00000-of-00001 (1).parquet")
val_df = val_df[val_df['text'].str.strip() != ""]
test_df = pd.read_parquet("/kaggle/input/mambaa/test-00000-of-00001 (3).parquet")
test_df = test_df[test_df['text'].str.strip() != ""]

def remove_special_characters(text):
    return re.sub(r"[^A-Za-z0-9\s.,!?']", '', text)

# Áp dụng hàm vào cột 'text'
train_df['text'] = train_df['text'].apply(remove_special_characters)
val_df['text'] = val_df['text'].apply(remove_special_characters)
test_df['text'] = test_df['text'].apply(remove_special_characters)

# Tạo Dataset từ DataFrame
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Tạo DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['text', '__index_level_0__'],
        num_rows: 23767
    })
    validation: Dataset({
        features: ['text', '__index_level_0__'],
        num_rows: 2461
    })
    test: Dataset({
        features: ['text', '__index_level_0__'],
        num_rows: 2891
    })
})


In [2]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Gán EOS token làm pad token.
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # Add special tokens and return token_type_ids=False since it's language modeling
    return tokenizer(
        examples["text"],
        padding=True,
        truncation=True,
        max_length=50,
        return_tensors=None,  # Don't force return tensors here - DataLoader will handle it
        return_special_tokens_mask=True
    )

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

# Remove the 'text' column and other unnecessary columns
tokenized_datasets = tokenized_datasets.remove_columns([col for col in tokenized_datasets["train"].column_names if col not in ["input_ids", "attention_mask"]])

print(tokenized_datasets)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/23767 [00:00<?, ? examples/s]

Map:   0%|          | 0/2461 [00:00<?, ? examples/s]

Map:   0%|          | 0/2891 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 23767
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2461
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2891
    })
})


In [4]:
pip install mamba_ssm triton

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached triton-3.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
Using cached triton-3.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209.5 MB)
  Created wheel for mamba_ssm: filename=mamba_ssm-2.2.4-cp310-cp310-linux_x86_64.whl size=323653202 sha256=c6edff068928b4ceacc6e820a163908c02294fe01dbcc05e95ed4530a5f81e77
  Stored in directory: /root/.cache/pip/wheels/aa/af/c7/fb77bfcd94bd3e052545033449d8c47dc97222d79c39c5bc67
Successfully built mamba_ssm
Note: you may need to restart the kernel to use updated packages.


In [8]:
from mamba_ssm.models.config_mamba import MambaConfig
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
import torch

# Cấu hình mô hình
config = MambaConfig(
    d_model=128,
    n_layer=6,
    vocab_size=tokenizer.vocab_size,
    ssm_cfg={},
    rms_norm=True,
    fused_add_norm=True,
    residual_in_fp32=True
)

# Khởi tạo mô hình
model = MambaLMHeadModel(config).to("cuda")

# Định nghĩa optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
batch_size=16

In [9]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

# Tạo DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
val_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    collate_fn=data_collator,
)


In [ ]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F
import math
# Vòng lặp huấn luyện
# Ensure GPUs are available
if torch.cuda.is_available() and torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs for training.")
    model = torch.nn.DataParallel(model)
else:
    print("Using a single GPU or CPU for training.")
num_epochs=20
# Move model to GPU(s)
model = model.to("cuda")
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to("cuda")
        outputs = model(input_ids)

        # Shift tokens
        shifted_input_ids = input_ids[:, 1:].clone()
        shifted_logits = outputs.logits[:, :-1, :].contiguous()

        loss = F.cross_entropy(
            shifted_logits.view(-1, shifted_logits.size(-1)),
            shifted_input_ids.reshape(-1),
            ignore_index=tokenizer.pad_token_id
        )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    train_perplexity = math.exp(avg_train_loss)
    print(f"Epoch {epoch+1}: Average training loss: {avg_train_loss}, Training perplexity: {train_perplexity}")

    # Đánh giá trên tập validation
    model.eval()
    total_eval_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch["input_ids"].to("cuda")
            outputs = model(input_ids)

            shifted_input_ids = input_ids[:, 1:].clone()
            shifted_logits = outputs.logits[:, :-1, :].contiguous()

            loss = F.cross_entropy(
                shifted_logits.view(-1, shifted_logits.size(-1)),
                shifted_input_ids.reshape(-1),
                ignore_index=tokenizer.pad_token_id
            )
            total_eval_loss += loss.item()

    avg_val_loss = total_eval_loss / len(val_dataloader)
    val_perplexity = math.exp(avg_val_loss)
    print(f"Epoch {epoch+1}: Average validation loss: {avg_val_loss}, Validation perplexity: {val_perplexity}")

# Lưu mô hình
model.save_pretrained("mamba_wikitext2")